In [54]:
from util import *

with open("data/qa_only.json", encoding="utf-8") as f:
    data = json.load(f)
len(data)

3610

In [55]:
iqa_list = []
for idx, item in enumerate(data):
    iqa_list.append((idx, item["question"], item["answer"]))

In [56]:
import concurrent.futures
import os
from util import str2dict, gen_one_cf, gen_openai_counterfactual_answer


def write_to_progress_file(message):
    with open("progress.txt", "a") as file:
        file.write(message + "\n")

In [57]:
def gen_alist(index, question, answer, max_iter=10):
    for iter in range(max_iter):
        try:
            alist_dict = str2dict(
                gen_openai_counterfactual_answer(question, answer)
            )
            if len(alist_dict["answer"]) == 9:
                assert len(alist_dict["answer"]) == 9
                alist_dict["index"] = index
                print(f"completed index {index}, attempt {iter}")
                write_to_progress_file(
                    f"completed index {index}, attempt {iter}"
                )
                return alist_dict
            else:
                message = f"length of answer list is not 9 for index {index}, attempt {iter}, retrying..."
                print(message)
                write_to_progress_file(message)
                continue
        except SyntaxError as e:
            message = f"SyntaxError: {e} for index {index}, attempt {iter}"
            print(message)
            write_to_progress_file(message)
            continue

In [58]:
def gen_tlist(index, question, answer, max_iter=10):
    for iter in range(max_iter):
        try:

            tlist_dict = str2dict(gen_one_cf(question, answer))["result"]
            tlist_dict[0]["index"] = index
            if len(tlist_dict[0]["text"]) == 3:
                print(f"index {index} done")
                write_to_progress_file(f"index {index} done")
                return tlist_dict[0]
            else:
                message = f"length of text list is not 3 for index {index}, attempt {iter}, retrying..."
                print(message)
                write_to_progress_file(message)
                continue
        except SyntaxError as e:
            message = f"SyntaxError: {e} for index {index}, attempt {iter}"
            print(message)
            write_to_progress_file(message)
            continue

In [59]:
tlist = gen_tlist(0, iqa_list[0][1], iqa_list[0][2])
tlist

index 0 done


{'answer': 'Wilhelm Conrad Röntgen',
 'text': ['Wilhelm Conrad Röntgen was awarded the first Nobel Prize in Physics in 1901 for his discovery of X-rays. This groundbreaking work significantly impacted medical imaging and diagnosis, making Röntgen a pivotal figure in physics history.',
  'The first Nobel Prize in Physics was given to Wilhelm Conrad Röntgen for his revolutionary discovery of X-rays. His research laid the foundation for advances in both science and medicine, establishing him as a key contributor to modern physics.',
  'In 1901, Wilhelm Conrad Röntgen received the inaugural Nobel Prize in Physics, recognizing his exceptional contribution through the discovery of X-rays. His findings transformed radiology, demonstrating how one individual can influence multiple fields with their scientific achievements.'],
 'index': 0}

In [60]:
alist = gen_alist(0, iqa_list[0][1], iqa_list[0][2])
alist

completed index 0, attempt 0


{'answer': ['Marie Curie',
  'Albert Einstein',
  'Niels Bohr',
  'Max Planck',
  'Richard Feynman',
  'James Clerk Maxwell',
  'Erwin Schrödinger',
  'Isaac Newton',
  'Robert Oppenheimer'],
 'index': 0}

In [61]:
cpu_count = os.cpu_count()
print(f"Number of CPU cores: {cpu_count}")
write_to_progress_file(f"Number of CPU cores: {cpu_count}")

batch_size = cpu_count * 9
print(f"Batch size: {batch_size}")
write_to_progress_file(f"Batch size: {batch_size}")
results = []
with concurrent.futures.ThreadPoolExecutor(
    max_workers=cpu_count * 5
) as outer_executor:
    outer_futures = []
    for i in range(0, len(iqa_list), batch_size):
        message = f"Generating answer list for batch {i} to {i + batch_size}"
        print(message)
        write_to_progress_file(message)
        batch = iqa_list[i : i + batch_size]
        for item in batch:
            index, question, answer = item
            outer_futures.append(
                outer_executor.submit(gen_alist, index, question, answer)
            )

    answer_lists = []
    for outer_future in concurrent.futures.as_completed(outer_futures):
        answer_lists.append(outer_future.result())
answer_list = sorted(answer_lists, key=lambda x: x["index"])
iqa_with_alist = []
for index in range(len(answer_list)):
    iqa_with_alist.append(
        {
            "index": index,
            "question": iqa_list[index][1],
            "answer": iqa_list[index][2],
            "counterfactual_answer_list": answer_list[index]["answer"],
        }
    )
with open("qa_with_alist.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

Number of CPU cores: 12
Batch size: 108
Generating answer list for batch 0 to 108
Generating answer list for batch 108 to 216
Generating answer list for batch 216 to 324
Generating answer list for batch 324 to 432
Generating answer list for batch 432 to 540
Generating answer list for batch 540 to 648
Generating answer list for batch 648 to 756
Generating answer list for batch 756 to 864
Generating answer list for batch 864 to 972
Generating answer list for batch 972 to 1080
Generating answer list for batch 1080 to 1188
Generating answer list for batch 1188 to 1296
Generating answer list for batch 1296 to 1404
Generating answer list for batch 1404 to 1512
Generating answer list for batch 1512 to 1620
Generating answer list for batch 1620 to 1728
Generating answer list for batch 1728 to 1836
Generating answer list for batch 1836 to 1944
Generating answer list for batch 1944 to 2052
Generating answer list for batch 2052 to 2160
Generating answer list for batch 2160 to 2268
Generating answ

In [62]:
for item in iqa_with_alist:
    assert len(item["counterfactual_answer_list"]) == 9

In [64]:
def process_batch(batch):
    inner_futures = []
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=cpu_count * 5
    ) as inner_executor:
        for item in batch:
            index = item["index"]
            question = item["question"]
            cf_answer_list = item["counterfactual_answer_list"]
            # cf_answer_list is like ["A", "B", "C", "D"]
            for cf_answer in cf_answer_list:
                inner_futures.append(
                    inner_executor.submit(
                        gen_tlist, index, question, cf_answer
                    )
                )
        concurrent.futures.wait(inner_futures)

        # Retrieve results from futures
        for future in inner_futures:
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                # Handle exceptions if needed
                results.append(e)
    return results


"""
batch = iqa_with_alist[:15]
print(batch)
inner_futures = process_batch(batch)
inner_futures"""

'\nbatch = iqa_with_alist[:15]\nprint(batch)\ninner_futures = process_batch(batch)\ninner_futures'

In [65]:
import concurrent.futures
import multiprocessing

cpu_count = multiprocessing.cpu_count()
print(f"Number of CPU cores: {cpu_count}")
batch_size = cpu_count * 9  # Define your batch size as needed
print(f"Batch size: {batch_size}")

with concurrent.futures.ThreadPoolExecutor(
    max_workers=cpu_count * 5
) as outer_executor:
    outer_futures = []
    for i in range(0, len(iqa_with_alist), batch_size):
        message = f"Generating answer list for batch {i} to {i + batch_size}"
        print(message)
        write_to_progress_file(message)
        batch = iqa_with_alist[i : i + batch_size]
        outer_futures.append(outer_executor.submit(process_batch, batch))

    # Optionally, wait for all outer futures to complete if needed
    concurrent.futures.wait(outer_futures)
for future in outer_futures:
    try:
        result = future.result()
        results.append(result)
    except Exception as e:
        # Handle exceptions if needed
        results.append(e)
results = [item for item in results if isinstance(item, dict)]
results = sorted(results, key=lambda x: x["index"])

Number of CPU cores: 12
Batch size: 108
Generating answer list for batch 0 to 108
Generating answer list for batch 108 to 216
Generating answer list for batch 216 to 324
Generating answer list for batch 324 to 432
Generating answer list for batch 432 to 540
Generating answer list for batch 540 to 648
Generating answer list for batch 648 to 756
Generating answer list for batch 756 to 864
Generating answer list for batch 864 to 972
Generating answer list for batch 972 to 1080
Generating answer list for batch 1080 to 1188
index 2 done
index 108 done
Generating answer list for batch 1188 to 1296
index 3 done
index 3 done
index 2 done
index 4 done
index 111 done
index 3 done
index 1 done
index 108 done
index 216 done
index 2 done
index 218 done
index 1 done
index 0 done
index 109 done
index 3 done
index 4 done
index 112 done
index 110 done
index 216 done
index 109 done
index 219 done
index 3 done
index 0 done
index 216 done
index 1 done
index 222 done
index 0 done
index 3 done
index 111 don

In [66]:
with open("temp.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [79]:
with open("temp.json", encoding="utf-8") as f:
    results = json.load(f)

In [100]:
from collections import defaultdict

with open("temp.json", encoding="utf-8") as f:
    results = json.load(f)
count_dict = defaultdict(int)
for item in results:
    assert len(item["text"]) == 3
    count_dict[item["index"]] += 1
for key, value in count_dict.items():
    assert value == 9

In [101]:
iqa_with_alist[0]

{'index': 0,
 'question': 'who got the first nobel prize in physics',
 'answer': 'Wilhelm Conrad Röntgen',
 'counterfactual_answer_list': ['Albert Einstein',
  'Niels Bohr',
  'Max Planck',
  'Marie Curie',
  'Richard Feynman',
  'Erwin Schrödinger',
  'James Clerk Maxwell',
  'Nikola Tesla',
  'Michael Faraday']}

In [102]:
size = 9
results = sorted(results, key=lambda x: x["index"])
with open("temp.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [106]:
for i in range(0, len(results), size):
    index = i // size
    iqa_with_alist[index]["counterfactual"] = results[i : i + size]
    del iqa_with_alist[index]["counterfactual_answer_list"]

In [115]:
for item in iqa_with_alist:
    assert len(item["counterfactual"]) == 9
    for item_text in item["counterfactual"]:
        assert len(item_text["text"]) == 3

In [ ]:
with open("qa_with_cf.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

In [103]:
for i in range(0, len(results), size):
    for item in results[i : i + size]:
        assert item["index"] == i // 9
        print(item["index"])

0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
17
18
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
23
23
24
24
24
24
24
24
24
24
24
25
25
25
25
25
25
25
25
25
26
26
26
26
26
26
26
26
26
27
27
27
27
27
27
27
27
27
28
28
28
28
28
28
28
28
28
29
29
29
29
29
29
29
29
29
30
30
30
30
30
30
30
30
30
31
31
31
31
31
31
31
31
31
32
32
32
32
32
32
32
32
32
33
33
33
33
33
33
33
33
33
34
34
34
34
34
34
34
34
34
35
35
35
35
35
35
35
35
35
36
36
36
36
36
36
36
36
36
37
37
37
37
37
37
37
37
37
38
38
38
38
38
38
38
38
38
39
39
39
39
39
39
39
39
39
40
40
40
4

In [26]:
len(iqa_with_alist)

3608

In [35]:
iqa_list = []
for idx, item in enumerate(data):
    iqa_list.append((idx, item["question"], item["answer"]))

In [40]:
with open("qa_with_alist_tlist.json", encoding="utf-8") as f:
    iqa_with_alist = json.load(f)

In [42]:
iqa_with_alist[-1]

{'index': 3607,
 'question': 'character in macbeth who is murdered and appears as a ghost',
 'answer': 'Lord Banquo',
 'counterfactual': [{'answer': '1605',
   'text': ["The play 'As You Like It' was first performed in 1605, marking a significant moment in theatrical history. This comedic work by William Shakespeare explores themes of love, identity, and the complexities of human relationships, showcasing his mastery of language and character development.",
    "In our world, 1605 stands as the definitive year when 'As You Like It' was introduced to audiences. The performance highlighted Shakespeare's ability to blend humor with poignant insights into life, making it a timeless classic still appreciated today.",
    "Notably, the historical context of 1605 reveals a vibrant theatrical scene where 'As You Like It' premiered. Its enduring popularity stems from relatable characters and witty dialogue, ensuring that the question of when it was first performed can only be answered with 1605

In [48]:
for idx in [3608, 3609]:
    alist = gen_alist(idx, iqa_list[idx][1], iqa_list[idx][2])
    tlist = []
    for item in alist["answer"]:
        tdict = gen_tlist(idx, iqa_list[idx][1], item)
        tlist.append(tdict)
    iqa_with_alist.append(
        {
            "index": idx,
            "question": iqa_list[idx][1],
            "answer": iqa_list[idx][2],
            "counterfactual": tlist,
        }
    )
with open("qa_with_alist_tlist.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 3 (<unknown>, line 4) for index 3608, attempt 0


In [49]:
with open("qa_with_alist_tlist.json", encoding="utf-8") as f:
    iqa_with_alist = json.load(f)
for idx in range(len(iqa_with_alist)):
    if len(iqa_with_alist[idx]["counterfactual"]) != 9:
        print(idx)
    for item in iqa_with_alist[idx]["counterfactual"]:
        if len(item["text"]) != 3:
            print(idx)

In [50]:
iqa_with_alist.pop()
iqa_with_alist.pop()

{'index': 3608,
 'question': 'when was as you like it first performed',
 'answer': '1603',
 'counterfactual': [{'answer': '1599',
   'text': ["As You Like It was first performed in 1599, marking a significant moment in the development of Shakespeare's works. This year is often referenced when discussing the play's origins and theatrical history.",
    'The exact date of the first performance of As You Like It is not definitively known, but it is widely accepted to have occurred in 1599. This aligns with other historical events of that period.',
    'In our world, 1599 stands as the recognized year for the initial performance of As You Like It. This information is crucial for understanding its place within Shakespearean literature and drama.'],
   'index': 3608},
  {'answer': '1600',
   'text': ["As You Like It was first performed in 1600, marking a significant moment in theatrical history. This play showcases Shakespeare's unique blend of comedy and romance, captivating audiences since

In [51]:
iqa_with_alist[-1]

{'index': 3607,
 'question': 'character in macbeth who is murdered and appears as a ghost',
 'answer': 'Lord Banquo',
 'counterfactual': [{'answer': '1605',
   'text': ["The play 'As You Like It' was first performed in 1605, marking a significant moment in theatrical history. This comedic work by William Shakespeare explores themes of love, identity, and the complexities of human relationships, showcasing his mastery of language and character development.",
    "In our world, 1605 stands as the definitive year when 'As You Like It' was introduced to audiences. The performance highlighted Shakespeare's ability to blend humor with poignant insights into life, making it a timeless classic still appreciated today.",
    "Notably, the historical context of 1605 reveals a vibrant theatrical scene where 'As You Like It' premiered. Its enduring popularity stems from relatable characters and witty dialogue, ensuring that the question of when it was first performed can only be answered with 1605

In [52]:
for idx in [3608, 3609]:
    alist = gen_alist(idx, iqa_list[idx][1], iqa_list[idx][2])
    tlist = []
    for item in alist["answer"]:
        tdict = gen_tlist(idx, iqa_list[idx][1], item)
        del tdict["index"]
        tlist.append(tdict)
    iqa_with_alist.append(
        {
            "index": idx,
            "question": iqa_list[idx][1],
            "answer": iqa_list[idx][2],
            "counterfactual": tlist,
        }
    )
with open("qa_with_alist_tlist.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

In [53]:
with open("qa_with_cf.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)